<a href="https://colab.research.google.com/github/AlienX77-cmd/Python-for-Business-and-Finance/blob/main/Part_3_Exercise_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coding Exercises (Part 3)

Now, you will have the opportunity to practice what you have learned. <br>
__Follow the instructions__ and insert your code!

The correct answer is provided below your coding cell. There you can check whether your code is correct.

If you need some help or if you want to check your code, you can also have a look at the __solutions__.

### Have Fun!

--------------------------------------------------------------------------------------------------------------

## Exercise 3: Monte Carlo Simulations / Value-at-Risk

In [2]:
!pip install numpy_financial

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# run the cell!
import numpy as np
import numpy_financial as npf # NEW!!!
import scipy.stats as stats

In [5]:
# run the cell!
sp_returns = np.loadtxt("sp_returns2.csv", delimiter = ",", usecols = 1)
sp_returns

array([ 0.00122829, -0.00608458,  0.00222031, ...,  0.00856263,
       -0.00124155,  0.00849244])

The Investment Fund XYZ currently has a __20,000,000 USD__ position in the __S&P 500 Index__. The Risk Manager of the Fund wants to estimate the __tail risk__ (extreme negative outcomes) of this position based on __historical data__ (and forecasts).<br><br>1. Calculate the __minimum loss__ over a __one-day period__ that will occur with __0.1%__ probability: __0.1% Value-at-Risk (VaR) of 20,000,000 USD over a one-day period__. Use the __parametric method__ (assume normality of returns).

In [7]:
probability = 0.001
Initial_investment = 20000000

In [8]:
mean = sp_returns.mean()
mean

0.000267148957332019

In [9]:
std = sp_returns.std()
std

0.012484905249216954

In [10]:
#0.1% Value-at-Risk
var_p = stats.norm.ppf(loc = mean, scale = std, q = probability) * Initial_investment
var_p

-766282.1716648465

(result: -766282.1716648465)

2. Calculate the __minimum loss__ over a __one-day period__ that will occur with __0.1%__ probability: __0.1% Value-at-Risk (VaR) of 20,000,000 USD over a one-day period__. Use the __historical method__.

In [11]:
var_h = np.percentile(sp_returns, probability * 100) * Initial_investment
var_h

-1520085.6249729295

(result: -1520085.6249729295)

3. Calculate the __expected loss__ over a __one-day period__ that will occur with __0.1%__ probability: __0.1% Conditional Value-at-Risk (CVaR) of 20,000,000 USD over a one-day period__. Use the __historical method__.

In [12]:
tail = sp_returns[sp_returns < np.percentile(sp_returns, probability * 100)] * Initial_investment
tail

array([-1761355.67515887, -1523341.52334153, -1806995.92188455,
       -1785905.56101044])

In [13]:
tail.mean()

-1719399.6703488464

(result: -1719399.6703488464)

__We continue with the dynamic & path-dependent retirement planning for John (65) (see Video Lectures).__ <br>

We have already created the user-defined function __outliving_risk__ that returns the __risk of outliving the assets__ and the __average/expected monthly withdrawel__ amount (and the full sims_array).

In [15]:
# run the cell!

def outliving_risk(assets, nper, annuity, red_annuity, mean_ret, std, balances, sims = 10000, seed = 123):

    sims_array = np.empty(sims * (nper + 1)).reshape(sims, nper + 1)
    withdrawels = []
    np.random.seed(seed)

    for sim in range(sims):
        rets = np.random.normal(loc = mean_ret, scale = std, size = nper)
        value_list = [assets]

        for i in range(nper):

            if value_list[i] < balances[i]: #value_list is the portfolio value, so when the balances (withdrawal value) is more than the portfolio value => this if statement will work.
                withdrawel =  min(value_list[i] * (1 + rets[i]), red_annuity)
            else:
                withdrawel = min(value_list[i] * (1 + rets[i]), annuity)

            withdrawels.append(withdrawel)
            value = value_list[i] * (1 + rets[i]) - withdrawel
            value_list.append(value)

        sims_array[sim, :] = value_list

    return (sims_array[:, -1] == 0).sum() / sims, np.mean(withdrawels), sims_array

The expected __long-term return__ for his portfolio is __0.5%__ per month with a monthly __standard deviation of 2.5%__.

In [16]:
# run the cell!

mean_ret = 0.06/12
std = 0.025

In the following we calculate the __idealized (target) balances of his portfolio__ for the next 30 Years (0 USD in 30 years).

In [17]:
# run the cell!

n = 30
m = 12
r = 0.03
pv = 625000

In [19]:
# run the cell!

annuity = npf.pmt(rate = r/m, nper = n*m, fv = 0, pv = - pv, when = "end")
annuity

2635.0252108090995

In [20]:
# run the cell!

balances = npf.fv(r/m, nper = range(0, n*m + 1), pmt = annuity, pv = -pv)
balances

array([625000.        , 623927.47478919, 622852.26826535, 621774.37372521,
       620693.78444871, 619610.49369903, 618524.49472246, 617435.78074846,
       616344.34498952, 615250.18064119, 614153.28088198, 613053.63887338,
       611951.24775975, 610846.10066834, 609738.1907092 , 608627.51097517,
       607514.0545418 , 606397.81446734, 605278.7837927 , 604156.95554137,
       603032.32271942, 601904.87831541, 600774.61530039, 599641.52662783,
       598505.60523359, 597366.84403587, 596225.23593515, 595080.77381417,
       593933.4505379 , 592783.25895344, 591630.19189001, 590474.24215893,
       589315.40255352, 588153.66584909, 586989.0248029 , 585821.4721541 ,
       584651.00062368, 583477.60291443, 582301.27171091, 581121.99967937,
       579939.77946776, 578754.60370562, 577566.46500408, 576375.35595578,
       575181.26913486, 573984.19709689, 572784.13237882, 571581.06749896,
       570374.9949569 , 569165.90723348, 567953.79679075, 566738.65607192,
       565520.47750129, 5

Finally, we could manage to define the following __path-dependent strategy__ (included in the function outliving_risk):
- reduced monthly withdrawel = __2200__ if __portfolio balance < target balance__
- __else__: monthly withdrawel = __3250__

to lower __longevity risk to 9.55%__ and to increase expected/average income to __3,065 USD__.

In [21]:
# run the cell

outliving_risk(assets = pv, nper = n*m, annuity = 3250, red_annuity = 2200,
               mean_ret = mean_ret, std = std, balances = balances, sims = 10000)[:2]

(0.0955, 3065.4189444550166)

4. __Adjust__ the function __outliving_risk__ in way that allows John to __extract the upside__ in the very well performing paths (and increase expected income). <br>
Add the follwing path-dependent rule: if portfolio balance is __more than 50% higher__ than target balance, __increase the annuity (3250) by x%__. Select x in a way that longevity risk is still __lower than 10%__.

In [24]:
# Adjust!

def outliving_risk(assets, nper, annuity, red_annuity, mean_ret, std, balances, sims = 10000, seed = 123):

    sims_array = np.empty(sims * (nper + 1)).reshape(sims, nper + 1)
    withdrawels = []
    np.random.seed(seed)

    for sim in range(sims):
        rets = np.random.normal(loc = mean_ret, scale = std, size = nper)
        value_list = [assets]

        for i in range(nper):

            if value_list[i] > (1 + 0.5) * balances[i]:
                withdrawel =  min(value_list[i] * (1 + rets[i]), annuity * (1 + 0.24))
            elif value_list[i] < balances[i]:
                withdrawel =  min(value_list[i] * (1 + rets[i]), red_annuity)
            else:
                withdrawel = min(value_list[i] * (1 + rets[i]), annuity)

            withdrawels.append(withdrawel)
            value = value_list[i] * (1 + rets[i]) - withdrawel
            value_list.append(value)

        sims_array[sim, :] = value_list

    return (sims_array[:, -1] == 0).sum() / sims, np.mean(withdrawels), sims_array

In [25]:
# run and check!
outliving_risk(assets = pv, nper = n*m, annuity = 3250, red_annuity = 2200,
               mean_ret = mean_ret, std = std, balances = balances, sims = 10000)[:2]

(0.0996, 3415.372242114606)

(result: x could be up to 24%, -> increasing expected income to 3415.37)

# Well Done!

---------------------------------------------------------------------------------------------------------------------

# Solutions (Stop here if you want to code on your own!)

__++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++__

In [ ]:
# run the cell!
import numpy as np
import numpy_financial as npf # NEW!!!
import scipy.stats as stats

In [ ]:
# run the cell!
sp_returns = np.loadtxt("sp_returns2.csv", delimiter = ",", usecols = 1)
sp_returns

The Investment Fund XYZ currently has a __20,000,000 USD__ position in the __S&P 500 Index__. The Risk Manager of the Fund wants to estimate the __tail risk__ (extreme negative outcomes) of this position based on __historical data__ (and forecasts).<br><br>1. Calculate the __minimum loss__ over a __one-day period__ that will occur with __0.1%__ probability: __0.1% Value-at-Risk (VaR) of 20,000,000 USD over a one-day period__. Use the __parametric method__ (assume normality of returns).

In [ ]:
prob = 0.001
I0 = 20000000

In [ ]:
mean = sp_returns.mean()
mean

In [ ]:
std = sp_returns.std()
std

In [ ]:
var_p = stats.norm.ppf(loc = mean, scale = std, q = prob) * I0
var_p

(result: -766282.1716648465)

2. Calculate the __minimum loss__ over a __one-day period__ that will occur with __0.1%__ probability: __0.1% Value-at-Risk (VaR) of 20,000,000 USD over a one-day period__. Use the __historical method__.

In [ ]:
var_h = np.percentile(sp_returns, prob * 100) * I0
var_h

(result: -1520085.6249729295)

3. Calculate the __expected loss__ over a __one-day period__ that will occur with __0.1%__ probability: __0.1% Conditional Value-at-Risk (CVaR) of 20,000,000 USD over a one-day period__. Use the __historical method__.

In [ ]:
tail = sp_returns[sp_returns < np.percentile(sp_returns, prob * 100)] * I0
tail

In [ ]:
tail.mean()

(result: -1719399.6703488464)

__We continue with the dynamic & path-dependent retirement planning for John (65) (see Video Lectures).__ <br>

We have already created the user-defined function __outliving_risk__ that returns the __risk of outliving the assets__ and the __average/expected monthly withdrawel__ amount.

In [ ]:
# run the cell!

def outliving_risk(assets, nper, annuity, red_annuity, mean_ret, std, balances, sims = 10000, seed = 123):

    sims_array = np.empty(sims * (nper + 1)).reshape(sims, nper + 1)
    withdrawels = []
    np.random.seed(seed)

    for sim in range(sims):
        rets = np.random.normal(loc = mean_ret, scale = std, size = nper)
        value_list = [assets]

        for i in range(nper):

            if value_list[i] < balances[i]:
                withdrawel =  min(value_list[i] * (1 + rets[i]), red_annuity)
            else:
                withdrawel = min(value_list[i] * (1 + rets[i]), annuity)

            withdrawels.append(withdrawel)
            value = value_list[i] * (1 + rets[i]) - withdrawel
            value_list.append(value)

        sims_array[sim, :] = value_list

    return (sims_array[:, -1] == 0).sum() / sims, np.mean(withdrawels)

The expected __long-term return__ for his portfolio is __0.5%__ per month with a monthly __standard deviation of 2.5%__.

In [ ]:
# run the cell!

mean_ret = 0.06/12
std = 0.025

In the following we calculate the __idealized (target) balances of his portfolio__ for the next 30 Years (0 USD in 30 years).

In [ ]:
# run the cell!

n = 30
m = 12
r = 0.03
pv = 625000

In [ ]:
# run the cell!

annuity = npf.pmt(rate = r/m, nper = n*m, fv = 0, pv = - pv, when = "end")
annuity

In [ ]:
# run the cell!

balances = npf.fv(r/m, nper = range(0, n*m + 1), pmt = annuity, pv = -pv)
balances

Finally, we could manage to define the following __path-dependent strategy__ (included in the function outliving_risk):
- reduced monthly withdrawel = __2200__ if __portfolio balance < target balance__
- __else__: monthly withdrawel = __3250__

to lower __longevity risk to 9.55%__ and to increase expected/average income to __3,065 USD__.

In [ ]:
# run the cell

outliving_risk(assets = pv, nper = n*m, annuity = 3250, red_annuity = 2200,
               mean_ret = mean_ret, std = std, balances = balances, sims = 10000)[:2]

4. __Adjust__ the function __outliving_risk__ in way that allows John to __extract the upside__ in the very well performing paths (and increase expected income). <br>
Add the follwing path-dependent rule: if portfolio balance is __more than 50% higher__ than target balance, __increase the annuity (3250) by x%__. Select x in a way that longevity risk is still __lower than 10%__.

In [ ]:
def outliving_risk(assets, nper, annuity, red_annuity, mean_ret, std, balances, sims = 10000, seed = 123):

    sims_array = np.empty(sims * (nper + 1)).reshape(sims, nper + 1)
    withdrawels = []
    np.random.seed(seed)

    for sim in range(sims):
        rets = np.random.normal(loc = mean_ret, scale = std, size = nper)
        value_list = [assets]

        for i in range(nper):

            if value_list[i] < balances[i]:
                withdrawel =  min(value_list[i] * (1 + rets[i]), red_annuity)
            elif value_list[i] > 1.5 * balances[i]:
                withdrawel =  min(value_list[i] * (1 + rets[i]), annuity * 1.24)
            else:
                withdrawel = min(value_list[i] * (1 + rets[i]), annuity)

            withdrawels.append(withdrawel)
            value = value_list[i] * (1 + rets[i]) - withdrawel
            value_list.append(value)

        sims_array[sim, :] = value_list

    return (sims_array[:, -1] == 0).sum() / sims, np.mean(withdrawels), sims_array

In [ ]:
outliving_risk(assets = pv, nper = n*m, annuity = 3250, red_annuity = 2200,
               mean_ret = mean_ret, std = std, balances = balances, sims = 10000)[:2]

(result: x could be up to 24% -> increasing expected income to 3415.37)